In [1]:
import shutup
shutup.please()

import gym
import src.environments.continuous.stock_trading  

import numpy as np
import pandas as pd 

In [2]:
def environment():
    env = gym.make('StockTradingEnvironment-v0',
                   use_technical_indicators= [
        "macd",
        "boll_ub",
        "boll_lb",
        "rsi_30",
        "cci_30",
        "dx_30",
        "close_30_sma",
        "close_60_sma",
    ])
    
    env.success_threshold =0.05 # 5%
    return env


In [3]:
env = environment()

In [4]:
env.load_dataset_by_index(0)
#env.df

In [5]:
env.success_threshold

0.05

In [6]:
state = env.reset()

In [7]:
state

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -1.15883102e-02, -7.84190720e-03,
        -1.35134723e-02, -1.29761027e-02,  1.81465704e-01,
         5.00000000e-01,  1.36300001e-01,  8.86032638e-02,
        -3.36551373e+03, -3.55492127e+03,  1.45888252e-02,
        -3.48753768e+03, -3.40262421e+03,  6.06336248e-02,
         5.87689287e+02,  5.38754966e-01,  0.00000000e+00,
         9.45000000e-01,  5.50000000e-02,  1.10550000e-01,
         4.53333333e-02,  8.97333333e-01,  5.73333333e-02,
         3.94416667e-01,  3.97916667e-01,  4.10000000e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  6.38525970e-03,  3.14666523e-03,
         4.64865087e-03, -1.20681984e-03, -1.93226567e-01,
         2.50000000e-01,  1.26400003e-01,  2.88514127e-02,
        -3.61227681e+04, -3.83755321e+04,  2.31190513e-

In [8]:
next_state = env.step([1])
next_state 

(array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  6.38525970e-03,  3.14666523e-03,
          4.64865087e-03, -1.20681984e-03, -1.93226567e-01,
          2.50000000e-01,  1.26400003e-01,  2.88514127e-02,
         -3.61227681e+04, -3.83755321e+04,  2.31190513e-01,
         -3.74992237e+04, -3.65438028e+04,  2.17428963e-01,
          6.57016512e+04,  5.67763088e-01,  5.25714286e-02,
          9.19428571e-01,  2.80000000e-02, -7.56714286e-02,
          2.85714286e-02,  8.75285714e-01,  9.60000000e-02,
          9.00257143e-01,  3.92500000e-01,  4.50000000e-01],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  2.20041089e-02,  1.19407742e-02,
          2.21382623e-02,  1.38851976e-02,  1.29904148e-01,
          0.00000000e+00,  1.26899996e-01,  7.29295868e-02,
          3.13824633e+03,  3.33823166e+

In [9]:
next_state[0][0] == state[1]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [10]:
from src.agents.actor_critic.a2c import A2CAgent

In [11]:
agent=A2CAgent(environment, epochs=1, actor_learning_rate=0.000025,critic_learning_rate=0.000025)
agent.load()
environment().success_threshold


[Errno 2] Unable to open file (unable to open file: name = 'storage/environments/StockTradingEnvironment/049eb1b62c63b64fe265373bb5f844ba/a2c-actor.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

[Errno 2] Unable to open file (unable to open file: name = 'storage/environments/StockTradingEnvironment/049eb1b62c63b64fe265373bb5f844ba/a2c-critic.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)




0.05

In [12]:
# agent.env.mode = "train"
# agent.learning_log.episodes = 0
# agent.learn(
#     timesteps=-1, 
#     log_every=1000,
#     success_threshold_lookback=1000,
#     success_strict=True,
    
# )

In [13]:
agent.env.mode = "test"
scores = []
success = 0

for i in range(2):
    state = agent.env.reset(visualize=True)
    score = 0
    reward = 0
    done = False
    initial_portfolio = agent.env.portfolio_value
    step = 0
    while not done:
        agent.env.render()
        #state = np.expand_dims(state, axis=0)
        action, action_onehot, prediction = agent.act(state)
        # Retrieve new state, reward, and whether the state is terminal
        next_state, reward, done, _ = agent.env.step(action)
        #print(action, reward, agent.env.portfolio_value)
        # Memorize (state, action, reward) for training
        #self.buffer.remember(np.expand_dims(state, axis=0), action_onehot, reward)
        # Update current state
        if done :
            if step < agent.env.window_size-1:
                success +=1
        step+=1
        state = next_state
        score += reward
    
    #print(score,initial_portfolio, agent.env.portfolio_value)
    scores.append(score)
    
print(np.mean(scores))
print(min(scores))
print(max(scores))
print(success)

C:\Users\filip\Development\GIT\tmp\reinforcement-learning\src\utils\trading_graph.py:164: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  img = np.fromstring(self.fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
Exception ignored in: <function TransformNode.set_children.<locals>.<lambda> at 0x000001D70A832710>
Traceback (most recent call last):
  File "C:\Users\filip\anaconda3\envs\ai_4\lib\site-packages\matplotlib\transforms.py", line 222, in <lambda>
    self, lambda _, pop=child._parents.pop, k=id(self): pop(k))
KeyboardInterrupt: 

KeyboardInterrupt



Error in callback <function flush_figures at 0x000001D77B9D40D0> (for post_execute):



KeyboardInterrupt



In [15]:
agent.env.close()

In [40]:
import os
import json


def log_environment_results_file(env_name,agent_hash,score):
    file_path = 'storage/environments/results.json'
    file_exists = os.path.exists(file_path)
    try:
        with open(file_path, 'r') as fp:
            data = json.load(fp)

    except IOError:
        print('File not found, will create a new one.')
        data = {}

    # do stuff with your data...
    with open(file_path, 'w') as fp:
        if agent_hash in data:
            if data[agent_hash] < score:
                data[agent_hash] = score
        else:
            data[agent_hash] = score
            
        json.dump(data, fp, indent=4)
        print(data)

In [44]:
log_environment_results_file('x','y',19)

{'y': 19}


In [42]:
d = {}

In [35]:
if 'x' in d:
    print(d['x'])

10


In [34]:
d['x'] = 10